# Toronto Data Segmentation and Clustering
By Jonathan Dietrich

## Part 1: Get the data and prepare it

In [1]:
import numpy as np
import pandas as pd

Put the toronto postal code table from the Wikipedia page into a Pandas Dataframe and remove the postal codes with no assigned burrow. 

In [2]:
table_url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
df_postal = pd.read_html(table_url, header=0)[0]
df_postal = df_postal[df_postal.Borough != 'Not assigned']
df_postal.reset_index(inplace=True)
del df_postal['index']
assert df_postal['Postal Code'].is_unique
df_postal.head()

,Postal Code,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [3]:
df_postal.shape

(103, 3)

## Part 2: Get the coordinates for the postal codes

In [4]:
# ! pip install geocoder
import geocoder

Get csv file with coordinates (geocoder API was not working).

In [5]:
df_coords = pd.read_csv('Geospatial_Coordinates.csv')
df_coords.set_index('Postal Code', inplace=True)
df_coords.head()

,Latitude,Longitude
Postal Code,,
M1B,43.806686,-79.194353
M1C,43.784535,-79.160497
M1E,43.763573,-79.188711
M1G,43.770992,-79.216917
M1H,43.773136,-79.239476


Put coordinates in df_postal by merging the dataframes.

In [6]:
df = df_postal.join(df_coords, on='Postal Code', how='left')
df

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
5,M9A,Etobicoke,"Islington Avenue, Humber Valley Village",43.667856,-79.532242
6,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
7,M3B,North York,Don Mills,43.745906,-79.352188
8,M4B,East York,"Parkview Hill, Woodbine Gardens",43.706397,-79.309937
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937


## Part 3: Explore and Cluster Neighborhoods

In [20]:
import json # library to handle JSON files

#! pip install geopy
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#! pip install folium
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


Only use boroughs that contain the word Toronto.

In [11]:
neighborhoods = df[df['Borough'].str.contains("Toronto")]
neighborhoods.reset_index(inplace=True)
del neighborhoods['index']
neighborhoods

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
1,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
3,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
4,M4E,East Toronto,The Beaches,43.676357,-79.293031
5,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306
6,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383
7,M6G,Downtown Toronto,Christie,43.669542,-79.422564
8,M5H,Downtown Toronto,"Richmond, Adelaide, King",43.650571,-79.384568
9,M6H,West Toronto,"Dufferin, Dovercourt Village",43.669005,-79.442259


Count boroughs and neighborhoods.

In [12]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(neighborhoods['Borough'].unique()),
        neighborhoods.shape[0]
    )
)

The dataframe has 4 boroughs and 39 neighborhoods.


Show neighborhoods on map.

In [17]:
# get mean latitude and longitude
latitude_mean = neighborhoods.Latitude.mean()
longitude_mean = neighborhoods.Longitude.mean()

# create map of New York using latitude and longitude values
map_toronto = folium.Map(location=[latitude_mean, longitude_mean], zoom_start=11)

# add markers to map
for lat, lng, borough, neighborhood in zip(neighborhoods['Latitude'], neighborhoods['Longitude'], neighborhoods['Borough'], neighborhoods['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

Next, we are going to start utilizing the Foursquare API to explore the neighborhoods and segment them.

Define Foursquare credentials and version in separate config file that is not in the git repository so my credentials are not published.

In [21]:
import foursquare_config